# **SIC Capstone Project**

*Supermarket Project*

### **Abstract Data Structures**

In [ ]:
class Queue:
    
    def __init__(self):
        self.queue = []

    def __str__(self):
        return "{}".format(self.queue)

    def is_empty(self):
        return len(self.queue) == 0

    def size(self):
        return len(self.queue)

    def enqueue(self, item):
        self.queue.append(item)

    def dequeue(self):
        return None if self.is_empty() else self.queue.pop(0)

    def peek_first(self):
        return None if self.is_empty() else self.queue[0]

    def peek_last(self):
        return None if self.is_empty() else self.queue[-1]

### **Custom Exceptions**

In [ ]:
class ClientNotFoundError(Exception):
    '''Triggers when the client_name is not in the agents dictionary'''
    def __init__(self, client_name):
        self.message = f"Client {client_name} not found"
        super().__init__(self.message)


class SupermarketNotFoundError(Exception):
    '''Triggers when the supermarket_name is not in the agents dictionary'''
    def __init__(self, supermarket_name):
        self.message = f"Supermarket {supermarket_name} not found"
        super().__init__(self.message)


class ClientNotInSupermarketError(Exception):
    '''Triggers when the client is not in a supermarket'''
    def __init__(self, client_name):
        self.message = f"Client {client_name} is not in a supermarket at this moment"
        super().__init__(self.message)

### **Class Hierarchy**

#### Agent

In [1]:
class Agent:
    """Clase base para todos los agentes en el sistema."""
    
    def __init__(self, name):
        self.name = name

    def describe(self):
        return f"Agent: {self.name}"

#### Client

In [2]:
from random import uniform
import json

class Client(Agent):
    """Clase que representa a un cliente que interactúa con el supermercado."""

    def __init__(self, name):
        super().__init__(name)
        self.status  = { 
            "is_in_supermarket": False, 
            "supermarket": "", 
            "client_has_paid": False, 
            "wallet": round(uniform(1000.0, 5000.0), 2),
            "is_checking_out": False,
            "is_waiting_in_line": False
        }
        self.shopping_cart = { 
                
        }


    def show_cart(self):
        return f"Shopping_cart: {json.dumps(self.shopping_cart)}"

    def info(self):
        return f'''
            Client name: {self.name}
            Status: {json.dumps(self.status)}
            shopping_cart: {json.dumps(self.shopping_cart)}
        '''         

#### Supermarket

In [ ]:
class Supermarket(Agent):
    """Clase que representa un supermercado"""

    def __init__(self, name):
        super().__init__(name)
        self.status = {
            "clients": [], 
            "checkout_line": Queue(),
            "is_open": True
        }
        self.inventory = {}
        
    def info(self):
        return f'''
            Supermarket name: {self.name}
            Status: {json.dumps(self.status)}
        '''  

    def add_product(self, product_name): 
        """ Añade un nuevo producto al supermercado """
        if product_name in self.inventory: #comprobamos si el producto se encuentra en el supermercado
            print(f'{product_name} already exist in {self.name}.')
        else:
            product_price = round(uniform(1.0, 50.0), 2)  #generamos un precio aleatorio entre 1.00 y 50.00
            product_stock = 0 # stock inicial por defecto
            self.inventory[product_name] = {
                "price" : product_price,
                "stock" : product_stock
            }
            print(f'Product {product_name} added to {self.name} with price: {product_price} and stock: {product_stock}.')


#### CitySimulation

In [ ]:
class CitySimulation:
    """Clase principal para gestionar la simulación de la ciudad."""
    
    def add_agent(self, agent_type, agent_name):
        """Añade un nuevo agente al sistema."""
        if agent_type == 'client':
            agents[agent_name] = Client(agent_name)
        elif agent_type == 'supermarket':
            agents[agent_name] = Supermarket(agent_name)
        print(f'{agent_type.capitalize()} {agent_name} added to the system.')

    def remove_agent(self, agent_name):
        """Elimina un agente del sistema."""
        if agent_name in agents:
            del agents[agent_name]
            print(f'Agent {agent_name} removed from the system.')
        else:
            print(f'Agent {agent_name} not found.')

    def list_agents(self):
        """Muestra todos los agentes en el sistema."""
        print("Current agents:")
        for agent in agents.values():
            print(agent.info())

    def help(self):
        """Muestra la lista de comandos disponibles."""
        print("""
            Available commands:
            - supermarket add <supermarket_name>: Agregar un nuevo supermercado al sistema.
            - supermarket add_product <supermarket_name> <product_name>: Agregar un producto al inventario del supermercado.
            - client add <client_name>: Agregar un cliente al sistema.
            - client move <client_name> <to_supermarket>: Mover a un cliente a un supermercado específico.
            - client pick_up <client_name> <product_name>: Permitir que un cliente recoja un producto.
            - supermarket show_clients <supermarket_name>: Mostrar la lista de clientes en el supermercado.
            - supermarket show_products <supermarket_name>: Mostrar la lista de productos en el supermercado.
            - supermarket restock <supermarket_name> <product_name> : Reabastecer un producto en el supermercado.
            - client buy <client_name> <product_name>: Comprar un producto y agregarlo a la cesta del cliente.
            - client show_cart <client_name>: Mostrar los productos en la cesta del cliente.
            - client checkout <client_name>: Preparar al cliente para pagar en caja.
            - supermarket show_clients_in_cash <supermarket_name>: Mostrar la lista de clientes en la caja.
            - supermarket serve_next_customer <supermarket_name>: Atender al siguiente cliente en la fila de caja.
            - client leave <client_name>: Permitir que un cliente salga del supermercado, siempre que haya pagado.
            - client leave_without_checkout <client_name>: Permitir que un cliente salga sin comprar, si su cesta está vacía.
            - client check_cart <client_name> <product_name>: Verificar si un producto está en la cesta del cliente.
            - supermarket set_open_close <supermarket_name> <open/close>: Abrir / Cerrar el supermercado si no hay clientes dentro.
            - supermarket show_inventory <supermarket_name>: Mostrar todo el inventario del supermercado.
            - client remove_from_cart <client_name> <product_name>: Permitir que el cliente retire un producto de su cesta.
            - client get_in_line <client_name>: Permitir que un cliente se ponga en la fila para pagar.
            """)            

    def command_loop(self):
        """Bucle principal para gestionar comandos del usuario."""
        print("Starting city simulation... Type 'q' to exit")
        while True:
            command = input('> ')
            if command == 'q':
                break
            self.process_command(command)

    def process_command(self, command):
        """Procesa los comandos ingresados por el usuario."""
        parts = command.split()
        if not parts:
            return
        
        cmd = parts[0]
        
        if cmd == '?':
            self.help()  # Llama al método de ayuda
            return
        
        # supermarket commands
        elif cmd == 'supermarket':

            #********************************************************
            #***** Completar con las funcionalidades requeridas *****
            #********************************************************


            ### falta hacer comprobaciones antes de añadirlo al diccionario
            if parts[1] == 'add':
                try:
                    _, _, supermarket_name = parts
                    self.add_agent("supermarket", supermarket_name)
                except ValueError:
                    print("Error: Invalid add command format. Use 'supermarket add <supermarket_name>'")

            elif parts[1] == 'add_product':
                pass

            elif parts[1] == 'show_clients':
                pass

            elif parts[1] == 'show_products':
                pass

            elif parts[1] == 'restock':
                pass

            elif parts[1] == 'show_clients_in_cart':
                pass

            elif parts[1] == 'serve_next_customer':
                pass

            elif parts[1] == 'set_open_close':
                pass

            elif parts[1] == 'show_inventory':
                pass



        # client commands
        elif cmd == 'client':

            ### falta hacer comprobaciones antes de añadirlo al diccionario
            if parts[1] == 'add':
                try:
                    _, _, client_name = parts
                    self.add_agent('client', client_name)
                except ValueError:
                    print("Error: Invalid add command format. Use 'client add <client_name>'")

            elif parts[1] == 'remove_client':
                try:
                    _, _, client_name = parts
                    self.remove_agent(client_name)
                except ValueError:
                    print("Error: Invalid remove_client command format. Use 'client remove_client <client_name>'")

            #********************************************************
            #***** Completar con las funcionalidades requeridas *****
            #********************************************************
            
            elif parts[1] == 'move':
                pass

            elif parts[1] == 'pick_up':
                pass

            elif parts[1] == 'buy':
                pass

            elif parts[1] == 'show_cart':
                pass

            elif parts[1] == 'checkout':
                pass

            elif parts[1] == 'leave':
                pass

            elif parts[1] == 'leave_without_checkout':
                pass

            elif parts[1] == 'check_cart':
                pass

            elif parts[1] == 'remove_from_cart':
                pass

            elif parts[1] == 'get_in_line':
                pass

        else:
            print("Unknown command. Type 'help' for a list of commands.")

### **Diccionario global para almacenar agentes**

In [ ]:
agents = {}

### **Main program**

In [ ]:
if __name__ == "__main__":
    simulation = CitySimulation()
    simulation.command_loop()